In [1]:
import os
import re
import json
import warnings
from time import time
import random
from multiprocessing import Pool
from tqdm.auto import tqdm
import fasttext
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
import csv
from gensim.utils import simple_preprocess

#### Основные настройки: seed, рабочий каталог.

In [2]:
SEED = 42
WORKDIR = '/home/ubuntu/gitrepo/KazanExpress/2/'
TEST_SIZE = 0.2

warnings.filterwarnings("ignore")
random.seed(SEED)
np.random.seed(seed=None) 

#### Преобразование входных данных

In [3]:
HTML_PATTERN = re.compile('<.*?>') 

def cleanhtml(raw_html):
    cleantext = re.sub(HTML_PATTERN, ' ', raw_html)
    return cleantext

def repr_field(s: object) -> str:
    s = cleanhtml(repr(s))
    s = s.replace('[', ' ').replace(']', ' ')
    s = s.replace('{', ' ').replace('}', ' ')
    s = s.replace("'", '')
    return s

def preprocess_text_field(field: str) -> str:
    field = json.loads(field)
    title = cleanhtml(str(field['title'])).replace('title', '')
    description = cleanhtml(str(field['description'])).replace('description', '')
    attributes = repr_field(field['attributes']).replace('attributes', '')
    custom_characteristics = repr_field(field['custom_characteristics']).replace('custom_characteristics', '')
    defined_characteristics = repr_field(field['defined_characteristics']).replace('defined_characteristics', '')
    filters = repr_field(field['filters']).replace('filters', '')
    return ". ".join([title, description, attributes, custom_characteristics, defined_characteristics, filters])

# Method for increasing the weight of the first words of title
def word_pyramid(string: str, min_n_words: int, max_n_words: int) -> list:
    result = []
    split = string.split(' ')
    for i in range(min_n_words, max_n_words+1):
        result += split[:i]
    return ' '.join(result)

# Predicting on a single input
def predict(document, model):
    return int(model.predict(document)[0][0][9:])


def mpredict(text):
    return int(predict(text, model))

In [4]:
# Read from parquet
data_full = pd.read_parquet(os.path.join(WORKDIR, 'row_data/train.parquet'))
# Drop unnecessary columns
data_full.drop(columns=['shop_id', 'rating'], inplace=True)
# Convert text fields
data_full.text_fields = data_full.text_fields.apply(preprocess_text_field)
# Convert "Sale"
data_full['sale'] = data_full['sale'].apply(lambda x: "Распродажа!" if x else "")  
data_full.fillna(value='', inplace=True)
# Concatenate to one string
data_full = data_full.assign(Document=[str(x) + '. ' + str(y) + '. ' + str(z) + '. ' + word_pyramid(x, 1, 7) + \
                    '. ' for x, y, z in zip(data_full['text_fields'], data_full['shop_title'], data_full['sale'])])

data_full = data_full.drop(columns=['text_fields', 'shop_title', 'sale']).reset_index(drop=True)
# Drop "product_id" column - only for train
data_full.drop(columns=['product_id'], inplace=True)
# Drop too rare values
drop_ids = set(data_full.category_id.value_counts()[data_full.category_id.value_counts() < 2].index)
print(f'Dropped {len(drop_ids)/data_full.shape[0]:.5f}% rows')
data_full = data_full[~data_full['category_id'].isin(drop_ids)]
# Trait/test split
data, data_valid = train_test_split(data_full.copy(), test_size=TEST_SIZE, random_state=SEED, shuffle=True, 
                                    stratify=data_full.category_id)
data.reset_index(drop=True, inplace=True)
data_valid.reset_index(drop=True, inplace=True)
# del data_full

Dropped 0.00004% rows


Преобразуем данные в формат, принимаемый FastText.

In [5]:
# Gensim simple preprocessing
data.Document = data.Document.apply(lambda x: ' '.join(simple_preprocess(x)))
data_valid.Document = data_valid.Document.apply(lambda x: ' '.join(simple_preprocess(x)))

# Add "__label__" mark for FastText
data.category_id = data.category_id.apply(lambda x: '__label__' + str(x))
data_valid.category_id = data_valid.category_id.apply(lambda x: '__label__' + str(x))

FastText принимает данные в виде текстовых файлов, поэтому сохраняем данные на диск.

In [6]:
# Saving the CSV file as a text file to train/test the classifier
data[['Document', 'category_id']].to_csv(os.path.join(WORKDIR, 'fasttext/train_ft.txt'), 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

data_valid[['Document', 'category_id']].to_csv(os.path.join(WORKDIR, 'fasttext/test_ft.txt'), 
                                               index = False, 
                                               sep = ' ',
                                               header = None, 
                                               quoting = csv.QUOTE_NONE, 
                                               quotechar = "", 
                                               escapechar = " ")

Обучаем модель.

In [10]:
# Training the fastText classifier
begin = time()
model = fasttext.train_supervised(os.path.join(WORKDIR, 'fasttext/train_ft.txt'),
                                  lr=0.65,                # learning rate [0.1]
                                  dim=100,               # size of word vectors [100]
                                  ws=5,                # size of the context window [5]
                                  epoch=70,             # number of epochs [5]
                                  neg=7,               # number of negatives sampled [5]
                                  wordNgrams=2, 
                                  minn=0, 
                                  maxn=0,
                                  minCount=1)
print(f'Time={time()-begin:.3f}')

Read 9M words
Number of words:  126840
Number of labels: 870
Progress:  99.9% words/sec/thread:  187779 lr:  0.000335 avg.loss:  0.172778 ETA:   0h 0m 0s  0.6% words/sec/thread:  210927 lr:  0.646121 avg.loss:  9.368999 ETA:   0h 2m14s

Time=153.854


100.0% words/sec/thread:  187752 lr: -0.000003 avg.loss:  0.172699 ETA:   0h 0m 0s
Progress: 100.0% words/sec/thread:  187751 lr:  0.000000 avg.loss:  0.172699 ETA:   0h 0m 0s


#### Смотрим на метрики:

In [11]:
# Evaluating performance on the entire train file
_, precision, recall = model.test(os.path.join(WORKDIR, 'fasttext/train_ft.txt')) 
f1_train = (2*precision*recall) / (precision+recall)

In [12]:
# Evaluating performance on the entire test file
_, precision, recall = model.test(os.path.join(WORKDIR, 'fasttext/test_ft.txt'))                      
f1_test = (2*precision*recall) / (precision+recall)

In [13]:
# Calculate weighted f1
begin = time()
data_valid_copy = data_valid.copy()
inputs = data_valid_copy.Document.tolist()

with Pool() as pool:
    outputs = pool.map(mpredict, inputs)

data_valid_copy["predicted_id"] = outputs
data_valid_copy.category_id = data_valid_copy.category_id.apply(lambda text: int(text[9:]))
weighted_f1 = f1_score(data_valid_copy.category_id.tolist(), 
                       data_valid_copy.predicted_id.tolist(), average='weighted')
print(f'Time={time()-begin:.3f}')

Time=0.796


In [14]:
print(f'Train F1={f1_train:.4f}')
print(f'Test F1={f1_test:.4f}') 
print(f'Weighted F1={weighted_f1:.4f}')

Train F1=0.9941
Test F1=0.8710
Weighted F1=0.8682


#### Кросс-валидация:

In [10]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

scores = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(data_full.Document, data_full.category_id)), total=5):
    data = data_full.iloc[train_index, :].copy()
    data_valid = data_full.iloc[test_index].copy()
    
    # Gensim simple preprocessing
    data.Document = data.Document.apply(lambda x: ' '.join(simple_preprocess(x)))
    data_valid.Document = data_valid.Document.apply(lambda x: ' '.join(simple_preprocess(x)))

    # Add "__label__" mark for FastText
    data.category_id = data.category_id.apply(lambda x: '__label__' + str(x))
    data_valid.category_id = data_valid.category_id.apply(lambda x: '__label__' + str(x))
    
    # Saving the CSV file as a text file to train/test the classifier
    data[['Document', 'category_id']].to_csv(os.path.join(WORKDIR, 'fasttext/train_ft.txt'), 
                                              index = False, 
                                              sep = ' ',
                                              header = None, 
                                              quoting = csv.QUOTE_NONE, 
                                              quotechar = "", 
                                              escapechar = " ")

    model = fasttext.train_supervised(os.path.join(WORKDIR, 'fasttext/train_ft.txt'),
                                  lr=0.65,                # learning rate [0.1]
                                  dim=100,               # size of word vectors [100]
                                  ws=5,                # size of the context window [5]
                                  epoch=70,             # number of epochs [5]
                                  neg=7,               # number of negatives sampled [5]
                                  wordNgrams=2, 
                                  minCount=1,
                                  verbose=0)
    # Calculate weighted f1
    data_valid_copy = data_valid.copy()
    inputs = data_valid_copy.Document.tolist()

    with Pool() as pool:
        outputs = pool.map(mpredict, inputs)

    data_valid_copy["predicted_id"] = outputs
    data_valid_copy.category_id = data_valid_copy.category_id.apply(lambda text: int(text[9:]))
    weighted_f1 = f1_score(data_valid_copy.category_id.tolist(), 
                           data_valid_copy.predicted_id.tolist(), average='weighted')
    scores.append(weighted_f1)
print(f'F1={np.array(scores).mean():.5f}')

  0%|          | 0/5 [00:00<?, ?it/s]

F1=0.86722


### Подбор гиперпараметров:

In [54]:
def try_parameters(lr=0.5, dim=80, ws=5, epoch=60, neg=7, wordNgrams=2):

    model = fasttext.train_supervised(os.path.join(WORKDIR, 'fasttext/train_ft.txt'),
                                  lr=lr,                # learning rate [0.1]
                                  dim=dim,               # size of word vectors [100]
                                  ws=ws,                # size of the context window [5]
                                  epoch=epoch,             # number of epochs [5]
                                  neg=neg,               # number of negatives sampled [5]
                                  wordNgrams=wordNgrams)
    
    data_valid_copy = data_valid.copy()
    data_valid_copy["predicted_id"] = data_valid_copy.Document.apply(lambda text: predict(text, model)).astype('int')
    data_valid_copy.category_id = data_valid_copy.category_id.apply(lambda text: int(text[9:]))
    weighted_f1 = f1_score(data_valid_copy.category_id.tolist(), 
                           data_valid_copy.predicted_id.tolist(), average='weighted')
    
    print(f'lr={lr}, dim={dim}, ws={ws}, epoch={epoch}, neg={neg}, wordNgrams={wordNgrams}')
    print(f'Weighted F1={weighted_f1:.4f}\n')


for lr in tqdm([0.4, 0.5]):
    for dim in tqdm([64, 80]):
        for epoch in [50, 60]:
            for wordNgrams in [1, 2, 3]:
                try_parameters(lr=lr, dim=dim, epoch=epoch, wordNgrams=wordNgrams)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Read 7M words
Number of words:  149031
Number of labels: 870
Progress: 100.0% words/sec/thread:  203691 lr:  0.000000 avg.loss:  0.289985 ETA:   0h 0m 0s  0.7% words/sec/thread:  210311 lr:  0.397066 avg.loss:  9.744263 ETA:   0h 0m40s


lr=0.4, dim=64, ws=5, epoch=50, neg=7, wordNgrams=1
Weighted F1=0.8535



Read 7M words
Number of words:  149031
Number of labels: 870
Progress: 100.0% words/sec/thread:  189827 lr: -0.000006 avg.loss:  0.432077 ETA:   0h 0m 0s  0.7% words/sec/thread:  202927 lr:  0.397167 avg.loss: 12.628447 ETA:   0h 0m42s  1.8% words/sec/thread:  197943 lr:  0.392652 avg.loss:  8.668149 ETA:   0h 0m42s  3.0% words/sec/thread:  198206 lr:  0.388054 avg.loss:  7.076039 ETA:   0h 0m42s  4.1% words/sec/thread:  197098 lr:  0.383556 avg.loss:  6.011496 ETA:   0h 0m42s  5.2% words/sec/thread:  195837 lr:  0.379127 avg.loss:  5.162292 ETA:   0h 0m41s  6.3% words/sec/thread:  195065 lr:  0.374693 avg.loss:  4.548378 ETA:   0h 0m41s  7.5% words/sec/thread:  195073 lr:  0.370175 avg.loss:  4.061322 ETA:   0h 0m41s  8.6% words/sec/thread:  194860 lr:  0.365695 avg.loss:  3.655100 ETA:   0h 0m40s 10.8% words/sec/thread:  194442 lr:  0.356764 avg.loss:  3.040461 ETA:   0h 0m39s 11.9% words/sec/thread:  194445 lr:  0.352260 avg.loss:  2.876940 ETA:   0h 0m39s 13.1% words/sec/thread:  1

...

lr=0.5, dim=80, ws=5, epoch=60, neg=7, wordNgrams=1
Weighted F1=0.8537



Read 7M words
Number of words:  149031
Number of labels: 870
Progress: 100.0% words/sec/thread:  153785 lr:  0.000000 avg.loss:  0.306872 ETA:   0h 0m 0s  1.7% words/sec/thread:  163556 lr:  0.491310 avg.loss:  7.438430 ETA:   0h 1m 2s 33.5% words/sec/thread:  155853 lr:  0.332337 avg.loss:  0.872609 ETA:   0h 0m44s 60.6% words/sec/thread:  156185 lr:  0.197117 avg.loss:  0.493094 ETA:   0h 0m26s 87.5% words/sec/thread:  154455 lr:  0.062638 avg.loss:  0.346547 ETA:   0h 0m 8s 98.6% words/sec/thread:  153876 lr:  0.007125 avg.loss:  0.310742 ETA:   0h 0m 0s


lr=0.5, dim=80, ws=5, epoch=60, neg=7, wordNgrams=2
Weighted F1=0.8568



Read 7M words
Number of words:  149031
Number of labels: 870
Progress: 100.0% words/sec/thread:  146186 lr:  0.000000 avg.loss:  0.394599 ETA:   0h 0m 0s  0.5% words/sec/thread:  159107 lr:  0.497687 avg.loss: 14.711055 ETA:   0h 1m 4s  1.6% words/sec/thread:  155119 lr:  0.491759 avg.loss:  8.975782 ETA:   0h 1m 5s  6.3% words/sec/thread:  151446 lr:  0.468577 avg.loss:  4.717517 ETA:   0h 1m 4s 15.2% words/sec/thread:  149808 lr:  0.424110 avg.loss:  2.362265 ETA:   0h 0m58s 41.0% words/sec/thread:  148173 lr:  0.294854 avg.loss:  0.915971 ETA:   0h 0m41s 59.9% words/sec/thread:  147360 lr:  0.200723 avg.loss:  0.642744 ETA:   0h 0m28s 66.4% words/sec/thread:  147127 lr:  0.167841 avg.loss:  0.581717 ETA:   0h 0m23s 71.0% words/sec/thread:  147218 lr:  0.144906 avg.loss:  0.546143 ETA:   0h 0m20s 73.0% words/sec/thread:  147146 lr:  0.135141 avg.loss:  0.532195 ETA:   0h 0m19s 81.2% words/sec/thread:  146792 lr:  0.094233 avg.loss:  0.479773 ETA:   0h 0m13s 83.5% words/sec/thread:  1

lr=0.5, dim=80, ws=5, epoch=60, neg=7, wordNgrams=3
Weighted F1=0.8519



In [14]:
for lr in tqdm([0.2, 0.25, 0.35]):
    for wordNgrams in tqdm([2, 3, 4, 5, 7]):
        for ws in [3, 5, 7]:
            try_parameters(lr=lr, dim=80, ws=ws, epoch=40, neg=7, wordNgrams=wordNgrams)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Read 7M words
Number of words:  149031
Number of labels: 870
Progress: 100.0% words/sec/thread:  149496 lr:  0.000000 avg.loss:  0.967649 ETA:   0h 0m 0s  0.7% words/sec/thread:  156409 lr:  0.198634 avg.loss: 16.353426 ETA:   0h 0m43s 22.6% words/sec/thread:  150420 lr:  0.154711 avg.loss:  3.314927 ETA:   0h 0m35s 42.9% words/sec/thread:  150279 lr:  0.114235 avg.loss:  1.972964 ETA:   0h 0m26s 52.3% words/sec/thread:  149974 lr:  0.095306 avg.loss:  1.670349 ETA:   0h 0m21s 59.2% words/sec/thread:  149814 lr:  0.081539 avg.loss:  1.506186 ETA:   0h 0m18s 0m 0s


lr=0.2, dim=80, ws=3, epoch=40, neg=7, wordNgrams=2
Weighted F1=0.8403, Train F1=0.9673, Test F1=0.8463 



Read 7M words
Number of words:  149031
Number of labels: 870
Progress: 100.0% words/sec/thread:  152856 lr:  0.000000 avg.loss:  1.019222 ETA:   0h 0m 0s


...

lr=0.35, dim=80, ws=3, epoch=40, neg=7, wordNgrams=7
Weighted F1=0.7968, Train F1=0.9458, Test F1=0.8074 



Read 7M words
Number of words:  149031
Number of labels: 870
Progress: 100.0% words/sec/thread:  118699 lr: -0.000012 avg.loss:  1.721961 ETA:   0h 0m 0s  6.8% words/sec/thread:  120477 lr:  0.326188 avg.loss:  9.523294 ETA:   0h 0m53s 12.9% words/sec/thread:  120118 lr:  0.304961 avg.loss:  7.185832 ETA:   0h 0m50s 0.301937 avg.loss:  6.952355 ETA:   0h 0m49s 15.1% words/sec/thread:  120009 lr:  0.297099 avg.loss:  6.593024 ETA:   0h 0m48s 33.5% words/sec/thread:  119317 lr:  0.232721 avg.loss:  3.994821 ETA:   0h 0m38s 48.8% words/sec/thread:  119122 lr:  0.179200 avg.loss:  3.037055 ETA:   0h 0m29s 51.8% words/sec/thread:  119204 lr:  0.168814 avg.loss:  2.907773 ETA:   0h 0m27s 0.000000 avg.loss:  1.721961 ETA:   0h 0m 0s


lr=0.35, dim=80, ws=5, epoch=40, neg=7, wordNgrams=7
Weighted F1=0.7959, Train F1=0.9453, Test F1=0.8068 



Read 7M words
Number of words:  149031
Number of labels: 870
Progress: 100.0% words/sec/thread:  118287 lr:  0.000000 avg.loss:  1.587188 ETA:   0h 0m 0s  0.5% words/sec/thread:  124798 lr:  0.348094 avg.loss: 17.751329 ETA:   0h 0m55s  1.4% words/sec/thread:  124158 lr:  0.344960 avg.loss: 15.637154 ETA:   0h 0m54s  2.3% words/sec/thread:  123423 lr:  0.341864 avg.loss: 14.313006 ETA:   0h 0m54s  3.2% words/sec/thread:  123206 lr:  0.338758 avg.loss: 13.242870 ETA:   0h 0m54s  4.1% words/sec/thread:  122490 lr:  0.335721 avg.loss: 12.378911 ETA:   0h 0m54s  4.9% words/sec/thread:  122029 lr:  0.332685 avg.loss: 11.534034 ETA:   0h 0m53s  5.8% words/sec/thread:  121600 lr:  0.329666 avg.loss: 10.736617 ETA:   0h 0m53s  6.7% words/sec/thread:  121375 lr:  0.326630 avg.loss:  9.939037 ETA:   0h 0m53s  7.5% words/sec/thread:  120986 lr:  0.323641 avg.loss:  9.323022 ETA:   0h 0m52s  8.4% words/sec/thread:  120732 lr:  0.320638 avg.loss:  8.856715 ETA:   0h 0m52s  9.3% words/sec/thread:  1

lr=0.35, dim=80, ws=7, epoch=40, neg=7, wordNgrams=7
Weighted F1=0.7967, Train F1=0.9456, Test F1=0.8070 

